<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/challenges/clock-decomposition/notebook-banner.jpg?inline=false" alt="Drawing" style="height: 400px;"/></p>

# Install packages 🗃

Please add all pacakage installations in this section

In [ ]:
!pip install numpy pandas lightgbm optuna scikit-Optimize imblearn

# Define preprocessing code 💻

The code that is common between the training and the prediction sections should be defined here. During evaluation, we completely skip the training section. Please make sure to add any common logic between the training and prediction sections here.

## Import common packages

Import packages that are common for training and prediction phases here.

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import log_loss
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

## Fill in missing values
We define a function to fill in the missing values with different approaches.

In [ ]:
# Define the function that fill in missing Value
def fillin_na(df):
    # fill in NA in column starts with missing digit with 1
    df.update(df.filter(regex='missing_digit', axis=1).fillna(1))

    # fill in NA in columns dist from cen & euc_dist_digit with mean
    for i in range(0, 12):
        df[str(i+1) + ' dist from cen'].fillna(df[str(i+1) +
                                                  ' dist from cen'].mean(), inplace=True)
        df['euc_dist_digit_' +
            str(i+1)].fillna(df['euc_dist_digit_' + str(i+1)].mean(), inplace=True)

    # fill in NA in column starts with area_digit / height_digit / width_digit with 0
    df.update(
        df.filter(regex=r'(area_digit|height_digit|width_digit)', axis=1).fillna(0))

    # fill value with -999
    ex_neg_col = ['variance_width', 'variance_height', 'variance_area', 'deviation_dist_from_mid_axis',
                  'between_axis_digits_angle_sum', 'between_axis_digits_angle_var', 'time_diff', 'centre_dot_detect']
    df[ex_neg_col] = df[ex_neg_col].fillna(-999)

    drop_col = ['between_digits_angle_ccw_sum', 'between_digits_angle_cw_sum',
                'between_digits_angle_ccw_var', 'sequence_flag_ccw', 'actual_hour_digit', 'actual_minute_digit']
    df[drop_col] = df[drop_col].fillna(-999)

    # fill value with 0
    fil_with_zero = ['number_of_digits', 'number_of_hands', 'hand_count_dummy',
                     'hour_hand_length', 'minute_hand_length', 'single_hand_length', 'clockhand_ratio', 'clockhand_diff',
                     'angle_between_hands', 'double_major', 'double_minor', 'vertical_dist', 'horizontal_dist']
    df[fil_with_zero] = df[fil_with_zero].fillna(0)

    # fill value with -1
    fil_with_one = ['deviation_from_centre', 'hour_proximity_from_11', 'minute_proximity_from_2',
                    'hour_pointing_digit', 'minute_pointing_digit', 'final_rotation_angle',
                    'top_area_perc', 'bottom_area_perc', 'left_area_perc', 'right_area_perc',
                    'between_digits_angle_cw_var', 'ellipse_circle_ratio', 'sequence_flag_cw', 'percentage_inside_ellipse']
    df[fil_with_one] = df[fil_with_one].fillna(-1)

    # Add missing value indicator
    df['time_diff_ind'] = df["time_diff"].apply(
        lambda x: 1 if math.isnan(x) else 0)
    df['centre_dot_detect_ind'] = df["centre_dot_detect"].apply(
        lambda x: 1 if math.isnan(x) else 0)

    # fill missing categorical variables
    df['intersection_pos_rel_centre'] = df['intersection_pos_rel_centre'].fillna(
        'Others')
    # OneHotEoncoding Categorical Variables
    df = pd.get_dummies(data=df, prefix=['intersec_dire'], columns=[
                        'intersection_pos_rel_centre'], drop_first=True)

    # drop unnecessary variables
    drop_col = ['between_digits_angle_ccw_sum', 'between_digits_angle_cw_sum',
                'between_digits_angle_ccw_var', 'sequence_flag_ccw', 'actual_hour_digit', 'actual_minute_digit']
    df = df.drop(drop_col, axis=1)

    return df

# Training phase ⚙️


## Load training data

In [ ]:
# Load the data
train = pd.read_csv(AICROWD_DATASET_PATH.replace("validation", "train"))
# Fill in the NA
train = fillin_na(train)

In [ ]:
train['diagnosis'].value_counts()

normal            31208
post_alzheimer     1149
pre_alzheimer       420
Name: diagnosis, dtype: int64

## Preprocessing - balance the data

### Simple undersample with random drop

In [ ]:
# Pre-process the training data

# Simple Undersampling
train_us = pd.concat([
    train.loc[train.diagnosis == 'pre_alzheimer'],
    train.loc[train.diagnosis == 'post_alzheimer'],
    train.loc[train.diagnosis == 'normal'].sample(frac=1/6),
]).reset_index().drop('index', axis=1)

# Seperate the target variable
train_y = train_us[['diagnosis']].values.ravel()

# Label encode the target variable
label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(np.array(train_y))

# Get the no cat data
cat_col = ['row_id', 'diagnosis']
train_no_cat = train_us.drop(cat_col, axis=1)

### Oversample with SMOTE and undersample with random drop

In [ ]:
''' Not good as Random Drop
# SMOTE
cat_col = ['row_id','diagnosis']
train_no_cat = train.drop(cat_col,axis=1)
train_y = train['diagnosis']

over = SMOTE(sampling_strategy={
    'normal':31208,
    'post_alzheimer':1149*5,
    'pre_alzheimer':420*5
})
under = RandomUnderSampler(sampling_strategy={
    'normal':int(31208/4), # keeping 25%
    'post_alzheimer':1149*5, # Keeping all of the samples we generated in the previous step
    'pre_alzheimer':420*5
})
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
train_no_cat, train_y = pipeline.fit_resample(train_no_cat, train_y)

train_y = train_y.values.ravel()

# Label encode the target variable
label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(np.array(train_y))
'''

" Not good as Random Drop\n# SMOTE\ncat_col = ['row_id','diagnosis']\ntrain_no_cat = train.drop(cat_col,axis=1)\ntrain_y = train['diagnosis']\n\nover = SMOTE(sampling_strategy={\n    'normal':31208,\n    'post_alzheimer':1149*5,\n    'pre_alzheimer':420*5\n})\nunder = RandomUnderSampler(sampling_strategy={\n    'normal':int(31208/4), # keeping 25%\n    'post_alzheimer':1149*5, # Keeping all of the samples we generated in the previous step\n    'pre_alzheimer':420*5\n})\nsteps = [('o', over), ('u', under)]\npipeline = Pipeline(steps=steps)\ntrain_no_cat, train_y = pipeline.fit_resample(train_no_cat, train_y)\n\ntrain_y = train_y.values.ravel()\n\n# Label encode the target variable\nlabel_encoder = LabelEncoder()\ntrain_y = label_encoder.fit_transform(np.array(train_y))\n"

## Feature Importantce Test

In [ ]:
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel

model = XGBClassifier()
fs = SelectFromModel(model)
fs.fit(X_train, y_train)
# X_train_fs = fs.transform(X_train)

feature_idx = fs.get_support()
feature_name = X_train.columns[feature_idx]

print(feature_name)

[16:39:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Index(['number_of_digits', 'missing_digit_4', 'missing_digit_7',
       'missing_digit_10', '10 dist from cen', '4 dist from cen',
       '6 dist from cen', '8 dist from cen', 'euc_dist_digit_3',
       'euc_dist_digit_9', 'euc_dist_digit_11', 'area_digit_6', 'area_digit_9',
       'height_digit_1', 'height_digit_10', 'width_digit_8',
       'between_axis_digits_angle_sum', 'between_axis_digits_angle_var',
       'between_digits_angle_cw_var', 'number_of_hands', 'single_hand_length',
       'angle_between_hands', 'minute_proximity_from_2', 'hour_pointing_digit',
       'count_defects', 'other_error', 'time_diff', 'centre_dot_detect',
       'intersec_dire_TR'],
      dtype='object')


## Hyper-parameter Tuning - LGB

In [ ]:
'''
import optuna


def objective(trial):
   
    X_train, X_valid, y_train, y_valid = train_test_split(train_no_cat,train_y, test_size=0.2)
    
    
    param = {
        "objective": "multiclass",
        'num_class': 3, 
        "metric": "multi_logloss",
        "boosting_type": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 64, 256),
        'min_data_in_leaf': trial.suggest_int("min_data_in_leaf", 20, 100),  
        'learning_rate': trial.suggest_float("learning_rate", 1e-8, 1.0, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'is_unbalance': True,
        'verbosity': -1
    }

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval  = lgb.Dataset(X_valid, y_valid, reference = lgb_train)
    model_lgb = lgb.train(param, lgb_train, 500, valid_sets=[lgb_eval], 
                    early_stopping_rounds=100)
    
    pred = model_lgb.predict(X_valid,num_iteration = model_lgb.best_iteration)
        
    logloss_lgb = log_loss(y_valid,pred)

    return logloss_lgb

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

'''

[I 2021-06-04 18:56:36,762] A new study created in memory with name: no-name-f8d51fe0-84c5-4164-a4a0-9ee9198e648b


[1]	valid_0's multi_logloss: 0.636887
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.636882
[3]	valid_0's multi_logloss: 0.636879
[4]	valid_0's multi_logloss: 0.636874
[5]	valid_0's multi_logloss: 0.63687
[6]	valid_0's multi_logloss: 0.636866
[7]	valid_0's multi_logloss: 0.636862
[8]	valid_0's multi_logloss: 0.636858
[9]	valid_0's multi_logloss: 0.636854
[10]	valid_0's multi_logloss: 0.636849
[11]	valid_0's multi_logloss: 0.636845
[12]	valid_0's multi_logloss: 0.636841
[13]	valid_0's multi_logloss: 0.636837
[14]	valid_0's multi_logloss: 0.636832
[15]	valid_0's multi_logloss: 0.636828
[16]	valid_0's multi_logloss: 0.636823
[17]	valid_0's multi_logloss: 0.63682
[18]	valid_0's multi_logloss: 0.636815
[19]	valid_0's multi_logloss: 0.636811
[20]	valid_0's multi_logloss: 0.636807
[21]	valid_0's multi_logloss: 0.636803
[22]	valid_0's multi_logloss: 0.636798
[23]	valid_0's multi_logloss: 0.636794
[24]	valid_0's multi_logloss: 0.636789
[25]	valid_0'

[210]	valid_0's multi_logloss: 0.636009
[211]	valid_0's multi_logloss: 0.636005
[212]	valid_0's multi_logloss: 0.636
[213]	valid_0's multi_logloss: 0.635996
[214]	valid_0's multi_logloss: 0.635992
[215]	valid_0's multi_logloss: 0.635988
[216]	valid_0's multi_logloss: 0.635984
[217]	valid_0's multi_logloss: 0.63598
[218]	valid_0's multi_logloss: 0.635976
[219]	valid_0's multi_logloss: 0.635972
[220]	valid_0's multi_logloss: 0.635968
[221]	valid_0's multi_logloss: 0.635964
[222]	valid_0's multi_logloss: 0.635959
[223]	valid_0's multi_logloss: 0.635955
[224]	valid_0's multi_logloss: 0.635951
[225]	valid_0's multi_logloss: 0.635947
[226]	valid_0's multi_logloss: 0.635943
[227]	valid_0's multi_logloss: 0.635938
[228]	valid_0's multi_logloss: 0.635934
[229]	valid_0's multi_logloss: 0.63593
[230]	valid_0's multi_logloss: 0.635925
[231]	valid_0's multi_logloss: 0.635921
[232]	valid_0's multi_logloss: 0.635916
[233]	valid_0's multi_logloss: 0.635912
[234]	valid_0's multi_logloss: 0.635908
[235]

[417]	valid_0's multi_logloss: 0.635144
[418]	valid_0's multi_logloss: 0.63514
[419]	valid_0's multi_logloss: 0.635136
[420]	valid_0's multi_logloss: 0.635132
[421]	valid_0's multi_logloss: 0.635128
[422]	valid_0's multi_logloss: 0.635124
[423]	valid_0's multi_logloss: 0.635119
[424]	valid_0's multi_logloss: 0.635115
[425]	valid_0's multi_logloss: 0.635111
[426]	valid_0's multi_logloss: 0.635107
[427]	valid_0's multi_logloss: 0.635102
[428]	valid_0's multi_logloss: 0.635098
[429]	valid_0's multi_logloss: 0.635094
[430]	valid_0's multi_logloss: 0.63509
[431]	valid_0's multi_logloss: 0.635086
[432]	valid_0's multi_logloss: 0.635082
[433]	valid_0's multi_logloss: 0.635078
[434]	valid_0's multi_logloss: 0.635073
[435]	valid_0's multi_logloss: 0.635069
[436]	valid_0's multi_logloss: 0.635065
[437]	valid_0's multi_logloss: 0.635061
[438]	valid_0's multi_logloss: 0.635057
[439]	valid_0's multi_logloss: 0.635053
[440]	valid_0's multi_logloss: 0.635049
[441]	valid_0's multi_logloss: 0.635044
[4

[I 2021-06-04 18:57:26,644] Trial 0 finished with value: 0.6347980135484809 and parameters: {'num_leaves': 127, 'min_data_in_leaf': 20, 'learning_rate': 1.1094208301678087e-05, 'feature_fraction': 0.7397792710562169, 'bagging_fraction': 0.9141531715837387, 'bagging_freq': 3}. Best is trial 0 with value: 0.6347980135484809.


...

[131]	valid_0's multi_logloss: 0.595387
[132]	valid_0's multi_logloss: 0.596578
Early stopping, best iteration is:
[32]	valid_0's multi_logloss: 0.540344
[1]	valid_0's multi_logloss: 0.555061
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.531387
[3]	valid_0's multi_logloss: 0.526222
[4]	valid_0's multi_logloss: 0.528989
[5]	valid_0's multi_logloss: 0.529569
[6]	valid_0's multi_logloss: 0.527074
[7]	valid_0's multi_logloss: 0.536097
[8]	valid_0's multi_logloss: 0.547044
[9]	valid_0's multi_logloss: 0.56629
[10]	valid_0's multi_logloss: 0.5732
[11]	valid_0's multi_logloss: 0.579123
[12]	valid_0's multi_logloss: 0.59164
[13]	valid_0's multi_logloss: 0.595177
[14]	valid_0's multi_logloss: 0.600791
[15]	valid_0's multi_logloss: 0.613692
[16]	valid_0's multi_logloss: 0.622085
[17]	valid_0's multi_logloss: 0.631605
[18]	valid_0's multi_logloss: 0.634708
[19]	valid_0's multi_logloss: 0.646639
[20]	valid_0's multi_logloss: 0.653715
[21]	valid_0's mu

[I 2021-06-04 19:07:32,153] Trial 98 finished with value: 0.5262217131298639 and parameters: {'num_leaves': 130, 'min_data_in_leaf': 60, 'learning_rate': 0.4904786538180361, 'feature_fraction': 0.7102417705061511, 'bagging_fraction': 0.6302414264372399, 'bagging_freq': 2}. Best is trial 57 with value: 0.4733056234728607.


[93]	valid_0's multi_logloss: 1.1912
[94]	valid_0's multi_logloss: 1.19662
[95]	valid_0's multi_logloss: 1.20383
[96]	valid_0's multi_logloss: 1.20918
[97]	valid_0's multi_logloss: 1.21347
[98]	valid_0's multi_logloss: 1.22086
[99]	valid_0's multi_logloss: 1.23015
[100]	valid_0's multi_logloss: 1.23998
[101]	valid_0's multi_logloss: 1.24305
[102]	valid_0's multi_logloss: 1.25002
[103]	valid_0's multi_logloss: 1.24814
Early stopping, best iteration is:
[3]	valid_0's multi_logloss: 0.526222
[1]	valid_0's multi_logloss: 0.619192
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.593076
[3]	valid_0's multi_logloss: 0.575673
[4]	valid_0's multi_logloss: 0.562598
[5]	valid_0's multi_logloss: 0.552699
[6]	valid_0's multi_logloss: 0.541963
[7]	valid_0's multi_logloss: 0.534363
[8]	valid_0's multi_logloss: 0.527157
[9]	valid_0's multi_logloss: 0.523511
[10]	valid_0's multi_logloss: 0.519412
[11]	valid_0's multi_logloss: 0.516088
[12]	valid_0's multi_log

[I 2021-06-04 19:07:33,599] Trial 99 finished with value: 0.507017441484504 and parameters: {'num_leaves': 83, 'min_data_in_leaf': 56, 'learning_rate': 0.13037608557443575, 'feature_fraction': 0.6790964170819194, 'bagging_fraction': 0.4585964569040078, 'bagging_freq': 2}. Best is trial 57 with value: 0.4733056234728607.


Number of finished trials: 100
Best trial:
  Value: 0.4733056234728607
  Params: 
    num_leaves: 253
    min_data_in_leaf: 54
    learning_rate: 0.05763827315209032
    feature_fraction: 0.7178896423652701
    bagging_fraction: 0.49186600291533183
    bagging_freq: 2


## LightGBM

In [ ]:
# K- fold
X_train = train_no_cat
y_train = pd.DataFrame(train_y)

skf = StratifiedKFold(n_splits=5, random_state=2021, shuffle=True)

In [ ]:
model_lgbs = []

params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'num_leaves': 128,
    'min_data_in_leaf': 100,
    'learning_rate': 0.04,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_gain_to_split': 0.2,
    'is_unbalance': True,
    'verbosity': -1
}

for fold, (itrain, ivalid) in enumerate(skf.split(X_train, y_train)):
    print("-"*40)
    print(f"Running for fold {fold}")
    lgb_train = lgb.Dataset(X_train.iloc[itrain], y_train.iloc[itrain])
    lgb_eval = lgb.Dataset(
        X_train.iloc[ivalid], y_train.iloc[ivalid], reference=lgb_train)
    model_lgb = lgb.train(params, lgb_train, 500, valid_sets=[lgb_eval],
                          early_stopping_rounds=100)

    model_lgbs.append(model_lgb)

----------------------------------------
Running for fold 0
[1]	valid_0's multi_logloss: 0.660085
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.646795
[3]	valid_0's multi_logloss: 0.634402
[4]	valid_0's multi_logloss: 0.623069
[5]	valid_0's multi_logloss: 0.613016
[6]	valid_0's multi_logloss: 0.604045
[7]	valid_0's multi_logloss: 0.59578
[8]	valid_0's multi_logloss: 0.588427
[9]	valid_0's multi_logloss: 0.581167
[10]	valid_0's multi_logloss: 0.574891
[11]	valid_0's multi_logloss: 0.569382
[12]	valid_0's multi_logloss: 0.564225
[13]	valid_0's multi_logloss: 0.559508
[14]	valid_0's multi_logloss: 0.55529
[15]	valid_0's multi_logloss: 0.55119
[16]	valid_0's multi_logloss: 0.547009
[17]	valid_0's multi_logloss: 0.543188
[18]	valid_0's multi_logloss: 0.540281
[19]	valid_0's multi_logloss: 0.536955
[20]	valid_0's multi_logloss: 0.533933
[21]	valid_0's multi_logloss: 0.531123
[22]	valid_0's multi_logloss: 0.528487
[23]	valid_0's multi_logloss: 0.

[16]	valid_0's multi_logloss: 0.565442
[17]	valid_0's multi_logloss: 0.562876
[18]	valid_0's multi_logloss: 0.560154
[19]	valid_0's multi_logloss: 0.557285
[20]	valid_0's multi_logloss: 0.554832
[21]	valid_0's multi_logloss: 0.552455
[22]	valid_0's multi_logloss: 0.549733
[23]	valid_0's multi_logloss: 0.547795
[24]	valid_0's multi_logloss: 0.545824
[25]	valid_0's multi_logloss: 0.544146
[26]	valid_0's multi_logloss: 0.542396
[27]	valid_0's multi_logloss: 0.540887
[28]	valid_0's multi_logloss: 0.539541
[29]	valid_0's multi_logloss: 0.537994
[30]	valid_0's multi_logloss: 0.536976
[31]	valid_0's multi_logloss: 0.535751
[32]	valid_0's multi_logloss: 0.534076
[33]	valid_0's multi_logloss: 0.533211
[34]	valid_0's multi_logloss: 0.532887
[35]	valid_0's multi_logloss: 0.532259
[36]	valid_0's multi_logloss: 0.531614
[37]	valid_0's multi_logloss: 0.531051
[38]	valid_0's multi_logloss: 0.530847
[39]	valid_0's multi_logloss: 0.530058
[40]	valid_0's multi_logloss: 0.529439
[41]	valid_0's multi_logl

[64]	valid_0's multi_logloss: 0.496459
[65]	valid_0's multi_logloss: 0.496417
[66]	valid_0's multi_logloss: 0.496357
[67]	valid_0's multi_logloss: 0.496653
[68]	valid_0's multi_logloss: 0.496984
[69]	valid_0's multi_logloss: 0.497371
[70]	valid_0's multi_logloss: 0.497067
[71]	valid_0's multi_logloss: 0.49695
[72]	valid_0's multi_logloss: 0.497125
[73]	valid_0's multi_logloss: 0.497137
[74]	valid_0's multi_logloss: 0.497385
[75]	valid_0's multi_logloss: 0.496998
[76]	valid_0's multi_logloss: 0.49748
[77]	valid_0's multi_logloss: 0.497838
[78]	valid_0's multi_logloss: 0.498374
[79]	valid_0's multi_logloss: 0.498533
[80]	valid_0's multi_logloss: 0.498608
[81]	valid_0's multi_logloss: 0.498976
[82]	valid_0's multi_logloss: 0.499512
[83]	valid_0's multi_logloss: 0.499287
[84]	valid_0's multi_logloss: 0.499392
[85]	valid_0's multi_logloss: 0.499265
[86]	valid_0's multi_logloss: 0.499358
[87]	valid_0's multi_logloss: 0.499579
[88]	valid_0's multi_logloss: 0.499851
[89]	valid_0's multi_loglos

[107]	valid_0's multi_logloss: 0.519377
[108]	valid_0's multi_logloss: 0.519615
[109]	valid_0's multi_logloss: 0.520142
[110]	valid_0's multi_logloss: 0.520189
[111]	valid_0's multi_logloss: 0.52049
[112]	valid_0's multi_logloss: 0.520748
[113]	valid_0's multi_logloss: 0.520735
[114]	valid_0's multi_logloss: 0.521064
[115]	valid_0's multi_logloss: 0.521567
[116]	valid_0's multi_logloss: 0.52195
[117]	valid_0's multi_logloss: 0.522192
[118]	valid_0's multi_logloss: 0.522619
[119]	valid_0's multi_logloss: 0.522509
[120]	valid_0's multi_logloss: 0.522825
[121]	valid_0's multi_logloss: 0.523515
[122]	valid_0's multi_logloss: 0.523795
[123]	valid_0's multi_logloss: 0.524226
[124]	valid_0's multi_logloss: 0.524303
[125]	valid_0's multi_logloss: 0.524596
[126]	valid_0's multi_logloss: 0.524709
[127]	valid_0's multi_logloss: 0.525304
[128]	valid_0's multi_logloss: 0.525437
[129]	valid_0's multi_logloss: 0.52574
[130]	valid_0's multi_logloss: 0.526112
[131]	valid_0's multi_logloss: 0.52652
[132

[146]	valid_0's multi_logloss: 0.507976
[147]	valid_0's multi_logloss: 0.508035
[148]	valid_0's multi_logloss: 0.508281
[149]	valid_0's multi_logloss: 0.50829
[150]	valid_0's multi_logloss: 0.50824
[151]	valid_0's multi_logloss: 0.508754
[152]	valid_0's multi_logloss: 0.509046
[153]	valid_0's multi_logloss: 0.508909
[154]	valid_0's multi_logloss: 0.509154
[155]	valid_0's multi_logloss: 0.509711
[156]	valid_0's multi_logloss: 0.509553
[157]	valid_0's multi_logloss: 0.509491
[158]	valid_0's multi_logloss: 0.509179
[159]	valid_0's multi_logloss: 0.509037
[160]	valid_0's multi_logloss: 0.509115
[161]	valid_0's multi_logloss: 0.509073
[162]	valid_0's multi_logloss: 0.509405
[163]	valid_0's multi_logloss: 0.509264
[164]	valid_0's multi_logloss: 0.509461
[165]	valid_0's multi_logloss: 0.509403
[166]	valid_0's multi_logloss: 0.509582
[167]	valid_0's multi_logloss: 0.509186
[168]	valid_0's multi_logloss: 0.509157
Early stopping, best iteration is:
[68]	valid_0's multi_logloss: 0.50009


## CatBoost

In [ ]:
from catboost import CatBoostClassifier

model_cats = []

for fold, (itrain, ivalid) in enumerate(skf.split(X_train, y_train)):
    print("-"*40)
    print(f"Running for fold {fold}")
    model_cat = CatBoostClassifier()
    model_cat.fit(X_train.iloc[itrain], y_train.iloc[itrain].values.ravel())
    model_cats.append(model_cat)

----------------------------------------
Running for fold 0
Learning rate set to 0.090553
0:	learn: 1.0230043	total: 37.3ms	remaining: 37.3s
1:	learn: 0.9632078	total: 71.8ms	remaining: 35.8s
2:	learn: 0.9132150	total: 107ms	remaining: 35.7s
3:	learn: 0.8735162	total: 140ms	remaining: 35s
4:	learn: 0.8342405	total: 176ms	remaining: 35.1s
5:	learn: 0.8051001	total: 212ms	remaining: 35.2s
6:	learn: 0.7808904	total: 259ms	remaining: 36.7s
7:	learn: 0.7570955	total: 308ms	remaining: 38.1s
8:	learn: 0.7347550	total: 357ms	remaining: 39.3s
9:	learn: 0.7187877	total: 400ms	remaining: 39.6s
10:	learn: 0.7033245	total: 458ms	remaining: 41.2s
11:	learn: 0.6889013	total: 513ms	remaining: 42.2s
12:	learn: 0.6760171	total: 559ms	remaining: 42.5s
13:	learn: 0.6638593	total: 610ms	remaining: 43s
14:	learn: 0.6541084	total: 648ms	remaining: 42.5s
15:	learn: 0.6421930	total: 681ms	remaining: 41.9s
16:	learn: 0.6329280	total: 714ms	remaining: 41.3s
17:	learn: 0.6240129	total: 754ms	remaining: 41.1s
18:	

163:	learn: 0.3760523	total: 6.46s	remaining: 32.9s
164:	learn: 0.3754442	total: 6.49s	remaining: 32.8s
165:	learn: 0.3746350	total: 6.52s	remaining: 32.8s
166:	learn: 0.3739211	total: 6.55s	remaining: 32.7s
167:	learn: 0.3732555	total: 6.58s	remaining: 32.6s
168:	learn: 0.3728828	total: 6.61s	remaining: 32.5s
169:	learn: 0.3721247	total: 6.64s	remaining: 32.4s
170:	learn: 0.3717597	total: 6.68s	remaining: 32.4s
171:	learn: 0.3708826	total: 6.71s	remaining: 32.3s
172:	learn: 0.3702142	total: 6.75s	remaining: 32.3s
173:	learn: 0.3691802	total: 6.78s	remaining: 32.2s
174:	learn: 0.3682577	total: 6.81s	remaining: 32.1s
175:	learn: 0.3672279	total: 6.84s	remaining: 32s
176:	learn: 0.3665319	total: 6.88s	remaining: 32s
177:	learn: 0.3656258	total: 6.92s	remaining: 32s
178:	learn: 0.3646672	total: 6.96s	remaining: 31.9s
179:	learn: 0.3639536	total: 6.99s	remaining: 31.8s
180:	learn: 0.3632578	total: 7.02s	remaining: 31.8s
181:	learn: 0.3623575	total: 7.05s	remaining: 31.7s
182:	learn: 0.3614

327:	learn: 0.2853464	total: 12.5s	remaining: 25.6s
328:	learn: 0.2851899	total: 12.5s	remaining: 25.6s
329:	learn: 0.2848787	total: 12.6s	remaining: 25.5s
330:	learn: 0.2842164	total: 12.6s	remaining: 25.5s
331:	learn: 0.2837754	total: 12.7s	remaining: 25.5s
332:	learn: 0.2836195	total: 12.7s	remaining: 25.4s
333:	learn: 0.2832092	total: 12.7s	remaining: 25.4s
334:	learn: 0.2829610	total: 12.8s	remaining: 25.4s
335:	learn: 0.2826120	total: 12.8s	remaining: 25.3s
336:	learn: 0.2819615	total: 12.9s	remaining: 25.3s
337:	learn: 0.2815633	total: 12.9s	remaining: 25.3s
338:	learn: 0.2812350	total: 12.9s	remaining: 25.2s
339:	learn: 0.2808535	total: 13s	remaining: 25.2s
340:	learn: 0.2806862	total: 13s	remaining: 25.2s
341:	learn: 0.2803399	total: 13.1s	remaining: 25.1s
342:	learn: 0.2798822	total: 13.1s	remaining: 25.1s
343:	learn: 0.2793290	total: 13.2s	remaining: 25.1s
344:	learn: 0.2790884	total: 13.2s	remaining: 25.1s
345:	learn: 0.2783599	total: 13.3s	remaining: 25.1s
346:	learn: 0.27

489:	learn: 0.2331212	total: 18s	remaining: 18.8s
490:	learn: 0.2329217	total: 18.1s	remaining: 18.7s
491:	learn: 0.2327396	total: 18.1s	remaining: 18.7s
492:	learn: 0.2324734	total: 18.1s	remaining: 18.6s
493:	learn: 0.2320911	total: 18.2s	remaining: 18.6s
494:	learn: 0.2319830	total: 18.2s	remaining: 18.6s
495:	learn: 0.2313619	total: 18.2s	remaining: 18.5s
496:	learn: 0.2313052	total: 18.3s	remaining: 18.5s
497:	learn: 0.2310770	total: 18.3s	remaining: 18.4s
498:	learn: 0.2307853	total: 18.3s	remaining: 18.4s
499:	learn: 0.2307248	total: 18.4s	remaining: 18.4s
500:	learn: 0.2305998	total: 18.4s	remaining: 18.3s
501:	learn: 0.2304443	total: 18.4s	remaining: 18.3s
502:	learn: 0.2300512	total: 18.4s	remaining: 18.2s
503:	learn: 0.2299148	total: 18.5s	remaining: 18.2s
504:	learn: 0.2296236	total: 18.5s	remaining: 18.1s
505:	learn: 0.2293197	total: 18.5s	remaining: 18.1s
506:	learn: 0.2290826	total: 18.6s	remaining: 18.1s
507:	learn: 0.2284463	total: 18.6s	remaining: 18s
508:	learn: 0.22

653:	learn: 0.1964120	total: 23.6s	remaining: 12.5s
654:	learn: 0.1962518	total: 23.6s	remaining: 12.4s
655:	learn: 0.1959378	total: 23.6s	remaining: 12.4s
656:	learn: 0.1958055	total: 23.7s	remaining: 12.3s
657:	learn: 0.1956109	total: 23.7s	remaining: 12.3s
658:	learn: 0.1951612	total: 23.7s	remaining: 12.3s
659:	learn: 0.1950006	total: 23.8s	remaining: 12.2s
660:	learn: 0.1948158	total: 23.8s	remaining: 12.2s
661:	learn: 0.1946595	total: 23.8s	remaining: 12.2s
662:	learn: 0.1943987	total: 23.9s	remaining: 12.1s
663:	learn: 0.1942014	total: 23.9s	remaining: 12.1s
664:	learn: 0.1941264	total: 23.9s	remaining: 12.1s
665:	learn: 0.1939930	total: 24s	remaining: 12s
666:	learn: 0.1938678	total: 24s	remaining: 12s
667:	learn: 0.1937526	total: 24s	remaining: 11.9s
668:	learn: 0.1934864	total: 24.1s	remaining: 11.9s
669:	learn: 0.1932080	total: 24.1s	remaining: 11.9s
670:	learn: 0.1926643	total: 24.2s	remaining: 11.8s
671:	learn: 0.1923377	total: 24.2s	remaining: 11.8s
672:	learn: 0.1921602	

...

229:	learn: 0.3359365	total: 7.79s	remaining: 26.1s
230:	learn: 0.3352496	total: 7.82s	remaining: 26s
231:	learn: 0.3350084	total: 7.85s	remaining: 26s
232:	learn: 0.3343967	total: 7.88s	remaining: 26s
233:	learn: 0.3339422	total: 7.92s	remaining: 25.9s
234:	learn: 0.3334347	total: 7.95s	remaining: 25.9s
235:	learn: 0.3327384	total: 7.97s	remaining: 25.8s
236:	learn: 0.3320960	total: 8.01s	remaining: 25.8s
237:	learn: 0.3313348	total: 8.05s	remaining: 25.8s
238:	learn: 0.3307521	total: 8.08s	remaining: 25.7s
239:	learn: 0.3302798	total: 8.11s	remaining: 25.7s
240:	learn: 0.3297169	total: 8.14s	remaining: 25.6s
241:	learn: 0.3293747	total: 8.16s	remaining: 25.6s
242:	learn: 0.3287396	total: 8.2s	remaining: 25.5s
243:	learn: 0.3284546	total: 8.23s	remaining: 25.5s
244:	learn: 0.3281737	total: 8.26s	remaining: 25.5s
245:	learn: 0.3274469	total: 8.29s	remaining: 25.4s
246:	learn: 0.3271273	total: 8.32s	remaining: 25.4s
247:	learn: 0.3261385	total: 8.35s	remaining: 25.3s
248:	learn: 0.32530

392:	learn: 0.2644230	total: 13.2s	remaining: 20.3s
393:	learn: 0.2639514	total: 13.2s	remaining: 20.3s
394:	learn: 0.2634735	total: 13.2s	remaining: 20.2s
395:	learn: 0.2629198	total: 13.2s	remaining: 20.2s
396:	learn: 0.2624189	total: 13.3s	remaining: 20.2s
397:	learn: 0.2621151	total: 13.3s	remaining: 20.1s
398:	learn: 0.2616755	total: 13.3s	remaining: 20.1s
399:	learn: 0.2614461	total: 13.4s	remaining: 20.1s
400:	learn: 0.2612097	total: 13.4s	remaining: 20s
401:	learn: 0.2608923	total: 13.4s	remaining: 20s
402:	learn: 0.2603296	total: 13.5s	remaining: 19.9s
403:	learn: 0.2598614	total: 13.5s	remaining: 19.9s
404:	learn: 0.2597586	total: 13.5s	remaining: 19.9s
405:	learn: 0.2594553	total: 13.6s	remaining: 19.8s
406:	learn: 0.2591168	total: 13.6s	remaining: 19.8s
407:	learn: 0.2588646	total: 13.6s	remaining: 19.8s
408:	learn: 0.2586823	total: 13.7s	remaining: 19.7s
409:	learn: 0.2584291	total: 13.7s	remaining: 19.7s
410:	learn: 0.2580330	total: 13.7s	remaining: 19.7s
411:	learn: 0.25

552:	learn: 0.2197852	total: 18.2s	remaining: 14.7s
553:	learn: 0.2196024	total: 18.2s	remaining: 14.7s
554:	learn: 0.2193096	total: 18.3s	remaining: 14.6s
555:	learn: 0.2190640	total: 18.3s	remaining: 14.6s
556:	learn: 0.2186865	total: 18.3s	remaining: 14.6s
557:	learn: 0.2184372	total: 18.3s	remaining: 14.5s
558:	learn: 0.2181955	total: 18.4s	remaining: 14.5s
559:	learn: 0.2179751	total: 18.4s	remaining: 14.5s
560:	learn: 0.2177645	total: 18.5s	remaining: 14.4s
561:	learn: 0.2175970	total: 18.5s	remaining: 14.4s
562:	learn: 0.2173456	total: 18.5s	remaining: 14.4s
563:	learn: 0.2170624	total: 18.5s	remaining: 14.3s
564:	learn: 0.2168345	total: 18.6s	remaining: 14.3s
565:	learn: 0.2166822	total: 18.6s	remaining: 14.3s
566:	learn: 0.2163972	total: 18.6s	remaining: 14.2s
567:	learn: 0.2161887	total: 18.7s	remaining: 14.2s
568:	learn: 0.2159115	total: 18.7s	remaining: 14.2s
569:	learn: 0.2155897	total: 18.7s	remaining: 14.1s
570:	learn: 0.2151950	total: 18.8s	remaining: 14.1s
571:	learn: 

713:	learn: 0.1860702	total: 23.4s	remaining: 9.37s
714:	learn: 0.1858515	total: 23.4s	remaining: 9.34s
715:	learn: 0.1856112	total: 23.5s	remaining: 9.31s
716:	learn: 0.1854501	total: 23.5s	remaining: 9.28s
717:	learn: 0.1853224	total: 23.5s	remaining: 9.24s
718:	learn: 0.1851069	total: 23.6s	remaining: 9.21s
719:	learn: 0.1849735	total: 23.6s	remaining: 9.18s
720:	learn: 0.1848357	total: 23.6s	remaining: 9.15s
721:	learn: 0.1846590	total: 23.7s	remaining: 9.12s
722:	learn: 0.1845153	total: 23.7s	remaining: 9.08s
723:	learn: 0.1841913	total: 23.7s	remaining: 9.05s
724:	learn: 0.1840082	total: 23.8s	remaining: 9.02s
725:	learn: 0.1839094	total: 23.8s	remaining: 8.98s
726:	learn: 0.1837991	total: 23.8s	remaining: 8.95s
727:	learn: 0.1835982	total: 23.9s	remaining: 8.92s
728:	learn: 0.1834491	total: 23.9s	remaining: 8.88s
729:	learn: 0.1833274	total: 23.9s	remaining: 8.85s
730:	learn: 0.1831673	total: 24s	remaining: 8.81s
731:	learn: 0.1830936	total: 24s	remaining: 8.78s
732:	learn: 0.18

875:	learn: 0.1614674	total: 28.7s	remaining: 4.06s
876:	learn: 0.1611544	total: 28.7s	remaining: 4.03s
877:	learn: 0.1610651	total: 28.8s	remaining: 4s
878:	learn: 0.1606748	total: 28.8s	remaining: 3.97s
879:	learn: 0.1605382	total: 28.8s	remaining: 3.93s
880:	learn: 0.1604644	total: 28.9s	remaining: 3.9s
881:	learn: 0.1604256	total: 28.9s	remaining: 3.87s
882:	learn: 0.1603076	total: 28.9s	remaining: 3.83s
883:	learn: 0.1602147	total: 29s	remaining: 3.8s
884:	learn: 0.1600353	total: 29s	remaining: 3.77s
885:	learn: 0.1599866	total: 29s	remaining: 3.73s
886:	learn: 0.1598798	total: 29.1s	remaining: 3.7s
887:	learn: 0.1597643	total: 29.1s	remaining: 3.67s
888:	learn: 0.1596304	total: 29.1s	remaining: 3.63s
889:	learn: 0.1593805	total: 29.2s	remaining: 3.6s
890:	learn: 0.1592677	total: 29.2s	remaining: 3.57s
891:	learn: 0.1591058	total: 29.2s	remaining: 3.54s
892:	learn: 0.1589823	total: 29.3s	remaining: 3.5s
893:	learn: 0.1588838	total: 29.3s	remaining: 3.47s
894:	learn: 0.1587606	tota

## Model selection

In [ ]:
# Load validation data
valid_data = pd.read_csv(AICROWD_DATASET_PATH)
valid_data = fillin_na(valid_data)
valid_no_cat = valid_data.drop(['row_id'], axis=1)

In [ ]:
# Load validation labels
valid_y = pd.read_csv(AICROWD_DATASET_PATH.replace(
    "validation", "validation_ground_truth"))
valid_y = valid_y['diagnosis']
nb_folds = 5  # skf.n_splits

In [ ]:
# lgb
lgb_preds = 0.0

for fold, model_lgb in enumerate(model_lgbs):
    print("-"*40)
    print(f"Running for fold {fold}")
    pred = model_lgb.predict(
        valid_no_cat, num_iteration=model_lgb.best_iteration)
    lgb_preds += pred/nb_folds

----------------------------------------
Running for fold 0
----------------------------------------
Running for fold 1
----------------------------------------
Running for fold 2
----------------------------------------
Running for fold 3
----------------------------------------
Running for fold 4
----------------------------------------
Running for fold 5
----------------------------------------
Running for fold 6
----------------------------------------
Running for fold 7
----------------------------------------
Running for fold 8
----------------------------------------
Running for fold 9


In [ ]:
# catBoost

cat_preds = 0.0

for fold, model_cat in enumerate(model_cats):
    print("-"*40)
    print(f"Running for fold {fold}")
    pred = model_cat.predict_proba(valid_no_cat)
    cat_preds += pred/nb_folds

----------------------------------------
Running for fold 0
----------------------------------------
Running for fold 1
----------------------------------------
Running for fold 2
----------------------------------------
Running for fold 3
----------------------------------------
Running for fold 4
----------------------------------------
Running for fold 5
----------------------------------------
Running for fold 6
----------------------------------------
Running for fold 7
----------------------------------------
Running for fold 8
----------------------------------------
Running for fold 9


In [ ]:
logloss_lgb = log_loss(valid_y, lgb_preds)
logloss_cat = log_loss(valid_y, cat_preds)

print("LGB logloss: " + str(logloss_lgb))
print("Catboost logloss: " + str(logloss_cat))

LGB logloss: 0.6306037118370114
Random Forest logloss: 0.5727656779163086
Catboost logloss: 0.5978673136784172


## Save the trained model
We decided to proceed with LightGBM.

In [ ]:
# # Save model
for i, model_lgb in enumerate(model_lgbs):
    model_filename = f'{AICROWD_ASSETS_DIR}/model_lgb_fold_{i}.pkl'
    joblib.dump(model_lgb, model_filename)

# Prediction phase 🔎

Please make sure to save the weights from the training section in your assets directory and load them in this section

In [ ]:
nb_folds = 5  # skf.n_splits
clfs = []

for fold in range(nb_folds):
    print("-"*40)
    print(f"Running for fold {fold}")
    model_filename = f'{AICROWD_ASSETS_DIR}/model_lgb_fold_{fold}.pkl'

    clf = joblib.load(model_filename)
    clfs.append(clf)

----------------------------------------
Running for fold 0
----------------------------------------
Running for fold 1
----------------------------------------
Running for fold 2
----------------------------------------
Running for fold 3
----------------------------------------
Running for fold 4


## Load test data

In [ ]:
test_data = pd.read_csv(AICROWD_DATASET_PATH)
test_data = fillin_na(test_data)
test_no_cat = test_data.drop(['row_id'], axis=1)

## Generate predictions

In [ ]:
preds = 0.0
for fold, clf in enumerate(clfs):
    print("-"*40)
    print(f"Running for fold {fold}")
    pred = clf.predict(test_no_cat, num_iteration=clf.best_iteration)
    preds += pred/nb_folds

----------------------------------------
Running for fold 0
----------------------------------------
Running for fold 1
----------------------------------------
Running for fold 2
----------------------------------------
Running for fold 3
----------------------------------------
Running for fold 4


In [ ]:
predictions = {
    "row_id": test_data["row_id"].values,
    "normal_diagnosis_probability": preds[:, 0],
    "post_alzheimer_diagnosis_probability": preds[:, 1],
    "pre_alzheimer_diagnosis_probability": preds[:, 2],
}

predictions_df = pd.DataFrame.from_dict(predictions)

## Save predictions 📨

In [ ]:
predictions_df.to_csv(AICROWD_PREDICTIONS_PATH, index=False)

# Submit to AIcrowd 🚀